In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")
songs = pd.read_csv("./input/songs.csv")
songs_info = pd.read_csv("./input/song_extra_info.csv")

# Let's take a look at the strucutre of the train, test, and song dataframes.

In [30]:
print("There are " + str(train.shape[0]) + " observations in the training data")

There are 7377418 observations in the training data


In [3]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


Die *target* Variable zeigt an ob innerhalb eines Monats nach dem ersten Hören des Songs der Song nochmals gespielt wurde. Also z.B.: <br>
Song wird an Tag 0 gehört, dann wieder an Tag 20 -> *target* = 1 <br>
Song wird an Tag 0 gehört, dass wieder an Tag 36 -> *target* = 0 <br>
Song wird an Tag 0 gehört, dann jeden Tag 5 mal -> *target* = 1 

In [31]:
test.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


In [5]:
songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


This shows us that the song information can be merged to the train and test data by joining on *song_id*

In [23]:
songs_info.head()

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


## Exploring Basic Statistics
##### Song statistics such as number of unique songs

In [6]:
print("Number of unique songs in train: " + str(train["song_id"].nunique()) + "     ",
      "    Number of unique songs in test: " +  str(test["song_id"].nunique()))

songs_in_train_and_test = np.intersect1d(train["song_id"].unique(), 
                                         test["song_id"].unique())
print("Number of unique songs that are in train and test: " + str(songs_in_train_and_test.shape[0]))

Number of unique songs in train: 359966          Number of unique songs in test: 224753
Number of unique songs that are in train and test: 164880


##### User statistics

In [7]:
print("Number of unique users in train: " + str(train["msno"].nunique()) + "     ",
     "Number of unqiue users in test: " + str(test["msno"].nunique()))

users_in_train_and_test = np.intersect1d(train["msno"].unique(), 
                                         test["msno"].unique())
print("Number of unique users that are in train and test: " + str(users_in_train_and_test.shape[0]))

Number of unique users in train: 30755      Number of unqiue users in test: 25131
Number of unique users that are in train and test: 21483


##### Artist statistics. 
First, however, we have to merge the song data to the train an test data.

// Excurs: Which of the following ways of merging the two dataframes is correct? I assume the second method (the left join) is correct. However, the first method  was used by a well rated [kaggle notebook](https://www.kaggle.com/kunstmord/exploring-the-songs/notebook)... so I want to be sure

In [9]:
train_merged_alt = train.merge(songs, on = "song_id")
print(train_merged_alt.shape)
train_merged = train.merge(songs, on = "song_id", how="left")
print(train_merged.shape)

(7377304, 12)
(7377418, 12)


In [14]:
test_merged = test.merge(songs, on = "song_id", how="left")

In [20]:
print("Number of unique artists in train: " + str(train_merged["artist_name"].nunique()) + "      ",
     "Number of unique artistis in test: " + str(test_merged["artist_name"].nunique()))

artists_in_train_and_test = np.intersect1d(train_merged['artist_name'].apply(str).unique(),
                                           test_merged['artist_name'].apply(str).unique())
print("Number of artists in train and test: " + str(artists_in_train_and_test.shape[0]))

Number of unique artists in train: 40582       Number of unique artistis in test: 27563
Number of artists in train and test: 21774


##### Language Statistics

In [17]:
print("Number of unique languages in train: " + str(train_merged["language"].nunique()) + "     ",
     "Number of unique languages in test: " + str(test_merged["language"].nunique()))

languages_in_train_and_test = np.intersect1d(train_merged["language"].unique(), test_merged["language"].unique())
print("Number of languages in train and test: " + str(languages_in_train_and_test.shape[0]))

Number of unique languages in train: 10      Number of unique languages in test: 10
Number of languages in train and test: 10


##### Genre statistics

In [21]:
print("Number of unique genres in train: " +  str(train_merged["genre_ids"].nunique()) + "     ",
    "Number of unique genres in test: " + str(test_merged["genre_ids"].nunique()))

genres_in_train_and_test = np.intersect1d(train_merged["genre_ids"].apply(str).unique(),
                                          test_merged["genre_ids"].apply(str).unique())
print("Number of languages in train and test: " + str(genres_in_train_and_test.shape[0]))

Number of unique genres in train: 572      Number of unique genres in test: 501
Number of languages in train and test: 466
